In [47]:
import pandas as pd
import numpy as np
import random

# Define constants for the dataset
regions = ["North America", "South America", "Europe", "Asia", "Africa", None]  # Include missing region
customer_segments = ["Enterprise", "Mid-Market", "SMB", None]  # Include missing segment
subscription_plans = ["Basic", "Premium", "Enterprise", None]  # Include missing plans
managers = ["Manager A", "Manager B", "Manager C", "Manager D", None]

# Generate sample data
np.random.seed(42)  # For reproducibility
num_records = 100

data = {
    "Customer_ID": [f"CUST{i:04d}" for i in range(1, num_records + 1)],
    "Region": np.random.choice(regions, num_records),
    "Customer_Segment": np.random.choice(customer_segments, num_records),
    "Subscription_Plan": np.random.choice(subscription_plans, num_records),
    "Monthly_Revenue": np.random.uniform(-1000, 50000, num_records),  # Include negative and high values
    "Monthly_Profit": np.random.uniform(-500, 20000, num_records),  # Include negative values
    "Churn_Flag": np.random.choice([0, 1, 2, None], num_records),  # Include invalid values (e.g., 2, None)
    "Renewal_Status": np.random.choice(["Renewed", "Churned", None], num_records),
    "Signup_Date": [None if random.random() < 0.1 else pd.Timestamp("2021-01-01") + pd.to_timedelta(random.randint(0, 1000), unit="d") for _ in range(num_records)],
    "Account_Manager": np.random.choice(managers, num_records)
}

# Create DataFrame
messy_data_df = pd.DataFrame(data)

# Save to CSV
messy_data_df.to_csv("messy_data.csv", index=False)

print("Messy data saved as 'messy_data.csv'")


Messy data saved as 'messy_data.csv'


In [49]:
messy_data_df.head()

,Customer_ID,Region,Customer_Segment,Subscription_Plan,Monthly_Revenue,Monthly_Profit,Churn_Flag,Renewal_Status,Signup_Date,Account_Manager
0,CUST0001,Asia,SMB,None,8512.444030,13900.923722,1,Renewed,2021-04-25,Manager A
1,CUST0002,Africa,Enterprise,None,1079.532219,6869.568600,2,None,2022-09-22,Manager A
2,CUST0003,Europe,Enterprise,Premium,29135.540103,5518.632807,None,Renewed,NaT,Manager A
3,CUST0004,Africa,Enterprise,Premium,33555.782454,16091.903687,0,Renewed,2023-06-30,None
4,CUST0005,Africa,SMB,Enterprise,-154.020725,16107.324591,0,None,NaT,Manager C


In [51]:
# ID missing values
missing_counts = messy_data_df.isnull().sum()
missing_percent = (missing_counts / len(messy_data_df))*100
print("Missing Values:\n", missing_counts)
print("Missing Percentage:\n", missing_percent)

Missing Values:
 Customer_ID           0
Region               17
Customer_Segment     23
Subscription_Plan    27
Monthly_Revenue       0
Monthly_Profit        0
Churn_Flag           25
Renewal_Status       31
Signup_Date          11
Account_Manager      24
dtype: int64
Missing Percentage:
 Customer_ID           0.0
Region               17.0
Customer_Segment     23.0
Subscription_Plan    27.0
Monthly_Revenue       0.0
Monthly_Profit        0.0
Churn_Flag           25.0
Renewal_Status       31.0
Signup_Date          11.0
Account_Manager      24.0
dtype: float64


In [70]:
# Handle categories by replacing null values with unknown

messy_data_df["Region"] = messy_data_df["Region"].fillna("Unknown")
messy_data_df["Customer_Segment"] = messy_data_df["Customer_Segment"].fillna("Unknown")
messy_data_df["Subscription_Plan"] = messy_data_df["Subscription_Plan"].fillna("Unknown")
messy_data_df["Renewal_Status"] = messy_data_df["Renewal_Status"].fillna("Unknown")

In [72]:
# Sort out churn flag issues
messy_data_df["Churn_Flag"] = messy_data_df["Renewal_Status"].apply(lambda x:1 if x == "Churned" else 0)
# Now here we've applied 0 to every one other than explicit churns, but what about those with null values?

In [74]:
missing_counts2 = messy_data_df.isnull().sum()
display(missing_counts2)

Customer_ID           0
Region                0
Customer_Segment      0
Subscription_Plan     0
Monthly_Revenue       0
Monthly_Profit        0
Churn_Flag            0
Renewal_Status        0
Signup_Date          11
Account_Manager      24
dtype: int64

In [76]:
# Filling in missing dates next. This one can be a little tricky and impact analysis further down the line
# In this instance we're going to replace the missing dates with the median (meaning that we can't really do a meaningful time series
# analysis)

messy_data_df["Signup_Date"] = pd.to_datetime(messy_data_df["Signup_Date"], errors="coerce")
median_date = messy_data_df["Signup_Date"].median()
messy_data_df["Signup_Date"] = messy_data_df["Signup_Date"].fillna(median_date)

In [82]:
# checking for and removing duplicates of key information

duplicates = messy_data_df.duplicated(subset=["Customer_ID"], keep=False)
print("Duplicated Customer IDs:\n")
display(messy_data_df[duplicates])

Duplicated Customer IDs:



,Customer_ID,Region,Customer_Segment,Subscription_Plan,Monthly_Revenue,Monthly_Profit,Churn_Flag,Renewal_Status,Signup_Date,Account_Manager


In [84]:
missing_counts3 = messy_data_df.isnull().sum()
display(missing_counts3)

Customer_ID           0
Region                0
Customer_Segment      0
Subscription_Plan     0
Monthly_Revenue       0
Monthly_Profit        0
Churn_Flag            0
Renewal_Status        0
Signup_Date           0
Account_Manager      24
dtype: int64

In [88]:
#okay, so heading into the last cleansing exercise I think as an exercise we'll identify and purge the data 
# where there is a missing Account Manager

missing_ams = messy_data_df["Account_Manager"].isnull().sum()
display(missing_ams)

24

In [92]:
cleaned_df = messy_data_df[messy_data_df["Account_Manager"].notnull()]
remaining_missing = cleaned_df["Account_Manager"].isnull().sum()
display(remaining_missing)

0

In [94]:
print(f"Original DataFrame Shape: {messy_data_df.shape}")
print(f"Cleaned DataFrame Shape: {cleaned_df.shape}")

Original DataFrame Shape: (100, 10)
Cleaned DataFrame Shape: (76, 10)


In [96]:
cleaned_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 76 entries, 0 to 99
Data columns (total 10 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   Customer_ID        76 non-null     object        
 1   Region             76 non-null     object        
 2   Customer_Segment   76 non-null     object        
 3   Subscription_Plan  76 non-null     object        
 4   Monthly_Revenue    76 non-null     float64       
 5   Monthly_Profit     76 non-null     float64       
 6   Churn_Flag         76 non-null     int64         
 7   Renewal_Status     76 non-null     object        
 8   Signup_Date        76 non-null     datetime64[ns]
 9   Account_Manager    76 non-null     object        
dtypes: datetime64[ns](1), float64(2), int64(1), object(6)
memory usage: 6.5+ KB


In [98]:
cleaned_df.to_csv("cleaned_df_for_upload.csv", index=False)